<img src='http://www.u.arizona.edu/~hammond/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2022<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Rachel 

***people I talked to about the assignment***: 

***

## Homework #4

Here are the imports. Please do not import anything else.

In [1]:
import graphviz,re
import pywrapfst as fst
import numpy as np

1. Using `pywrapfst`, create a symbol table for the (lowercase) English alphabet plus $\epsilon$ and `#`. (The second symbol added should be `#`.)

In [2]:
st = fst.SymbolTable()
leslettres = ['<epsilon>', '#', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
for letter in leslettres:
    st.add_symbol(letter)
#raise NotImplementedError()

In [3]:
assert st.num_symbols() == 28

In [4]:
assert st.find(0) == '<epsilon>'

In [5]:
assert st.find(1) == '#'

2. We'll now build a character *bigram model* based on the *Alice in Wonderland* text. The following function should strip the Gutenberg information at the beginning and end of the file, convert everything to lowercase, convert everything except lowercase letters to space, and return a list of lowercase words.

In [1]:
def getwords(alice):
    '''tokenizes the Alice text
    args:
        alice: the location of the Gutenberg alice file
    returns
        a list of lowercase words
    '''
    f = open(alice)
    book = f.read()
    f.close()
    lines = book.split ('\n')
    tokens = []
    count = 0
    for line in lines:
        this_line = line.lower().replace('_', ' ').strip().replace('ù', 'u')
        if count == 0:
            end = re.findall("evidence$", this_line)
            if end:
                count += 1
                continue
            else:
                continue
        if count == 1:
            order = r'[0-9]'
            negnumbers = re.sub(order, '', this_line)
            words = re.compile(r'\w+(?:\w+)?')
            final = words.findall(negnumbers)

         #   print(final1)
            tokens.extend(final)
        if this_line == "the end":
            count += 1
        if count == 2:
            continue
  #  print(tokens)
    return tokens

    
    #raise NotImplementedError()

In [7]:
words = getwords('alice.txt')
assert words[:12] == [
    'chapter', 'i', 'down', 'the', 'rabbit', 'hole',
    'alice', 'was', 'beginning', 'to', 'get', 'very'
]

In [8]:
assert words[-5:] == ['happy', 'summer', 'days', 'the', 'end']

In [9]:
assert len(words) == 27336

3. We now write a function that will return a list of counts for letter unigrams and letter bigrams. You'll need to pad each word with `#` on each side before doing this.

In [10]:
def getcounts(wds):
    '''get letter unigram and bigram counts from a list of words
    args:
        words: a list of words
    returns
        unigrams: a dictionary from letters to counts
        bigrams: a dictionary from letter pairs to counts
    '''
    newlist = []
    bigram_dict = {}
    for item in wds:
        words = "#" + item + "#"
        newlist.append(words)
    letters = '#abcdefghijklmnopqrstuvwxyz'
    unigram_dict = dict.fromkeys(letters,0)
    for item in newlist:
        for char in range(len(item)):
         #   print(item[char])
            unigram_dict[item[char]] += 1
        for char in range(len(item[:-1])): 
            bigram = item[char] + item[char+1]
          #  print (bigram)
            if bigram in bigram_dict:
                bigram_dict[bigram] += 1
            else:
                bigram_dict[bigram] = 1
   # print(bigram_dict)
    return unigram_dict, bigram_dict
        
  #  print (unigram_dict)
   # raise NotImplementedError()

In [11]:
ugs,bgs = getcounts(words)
assert len(ugs) == 27

In [12]:
assert len(bgs) == 427

In [13]:
assert bgs['ab'] == 214

In [14]:
assert ugs['#'] == 54672

In [15]:
assert ugs['q'] == 209

4. Let's now write a function that takes our unigram and bigram counts and creates a dictionary of log conditional probabilities.

In [16]:
def makecondprobs(unigrams,bigrams):
    '''calculate conditional probabilities
    args:
        unigrams: dictionary of unigram counts
        bigrams: dictionary of bigram counts
    returns:
        a dictionary of log conditional probabilities
    '''
    sumuni= 0 #sum of unigram couts
    sumbi= 0 #sum of bigram counts
    for value in unigrams.values():
        sumuni += value
    for values in bigrams.values():
        sumbi += values
    probAB = {}
    probA = {}
    demprobslog = {}
    for key,value in bigrams.items():
        unicount = unigrams.get(key[0]) #pulls the value from unigram of first letter
        condprob = (value/unicount)
        logcp = (np.log(condprob))
        demprobslog[key] = logcp
    return demprobslog

    #raise NotImplementedError()

In [17]:
cp = makecondprobs(ugs,bgs)
assert len(cp) == 427

In [18]:
assert np.isclose(cp['ab'],-3.71,atol=.01)

In [19]:
total = 0
for bigram in cp:
    if bigram[0] == 'a':
        total += np.exp(cp[bigram])
assert np.isclose(total,1.0,atol=.0001)

5. We now create a WFSA for this model using the symbol table you created in question #1 and the dictionary we just created. The state geometry is critical here. There will be a start state, which you can think of as the `#` on the left. From there, the first arc takes you to various letter states. The probabilities of those arcs will be the conditional probabilities of the different letters based on the initial `#`. Now from each letter state you can go to any other letter state or the final `#`. In each case, the relevant probability is the conditional probability of the second letter/`#` based on the first letter. The second `#` state is the only final state. Note that there will be no arc corresponding to the first `#`.

In [20]:
def makeWFSA(tab,dic):
    '''create a WFSA that encodes a bigram model
    args:
        tab: a symbol table including all letters
            and boundary markers
        dic: a dictionary of the conditional probabilities
    returns:
        a WFSA
    '''
   # st = fst.SymbolTable()  
    
    c1 = fst.Compiler(
        isymbols =tab,
        osymbols =tab,
        keep_isymbols =True,
        keep_osymbols=True,
        arc_type='log'
        )
   
     #create states
    states = {'<epsilon>':0, '#':1, 'a':2, 'b':3, 'c':4, 'd':5, 'e':6, 'f':7, 'g':8, 'h':9, 'i':10, 'j':11, 'k':12, 'l':13, 'm':14, 'n':15, 'o':16, 'p':17, 'q':18, 'r':19, 's':20, 't':21, 'u':22, 'v':23, 'w':24, 'x':25, 'y':26, 'z':27}
    for letter in dic:
        firststate = states[letter[0]]
        if letter[0] == "#":
            firststate = 0
   #         print(firststate)
        secondstate = states[letter [1]]
 #       c1.write("0 0 <epsilon> <epsilon> 0")
  #      print(str(firststate) + " " + str(secondstate) + " " + letter[1] + ' ' + letter[1] + ' ' + str(dic[letter]))
        c1.write(str(firststate) + " " + str(secondstate) + " " + letter[1] + ' ' + letter[1] + ' ' + str(dic[letter]))
    c1.write('1')
    f = c1.compile()
    return f

In [21]:
wfsa = makeWFSA(st,cp)
assert type(wfsa) == fst.MutableFst

In [22]:
assert wfsa.num_states() == 28

In [23]:
c2 = fst.Compiler(
    isymbols=st,
    osymbols=st,
    keep_isymbols=True,
    keep_osymbols=True,
    arc_type='log'
)
c2.write('0 1 h h')
c2.write('1 2 a a')
c2.write('2 3 t t')
c2.write('3 4 # #')
c2.write('4')
f2 = c2.compile()
f3 = fst.compose(f2,wfsa)
assert f3.num_states() == 5

In [24]:
f3s = str(f3).split('\n')[:-2]
val = np.exp(sum([float(s.split('\t')[-1]) for s in f3s]))
assert np.isclose(val,0.00019,atol=.001)